## 辨識

In [1]:
import os
HOME = os.getcwd()
print (HOME)

C:\Users\Admin\Desktop\yolov9


In [2]:
%cd {HOME}/yolov9

C:\Users\Admin\Desktop\yolov9\yolov9


In [ ]:
import sys
import json
import cv2
import numpy as np
import pyautogui
import subprocess
import time
import os
import glob
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout, QPushButton, QHBoxLayout
from PyQt5.QtGui import QPixmap, QImage
from PyQt5.QtCore import Qt

# 設定截圖範圍 (手動設定座標)
X1, Y1, X2, Y2 = 0, 0, 1820, 1080  # 左上角 (0,0) 到 右下角 (1820,1080)

particle_data = []
class ParticleSelector(QWidget):
    def __init__(self):
        super().__init__()
        self.image = None  # 初始沒有影像
        self.particle_data = []  # 存放辨識結果
        self.initUI()
        
        # 設定視窗獲取焦點，以便接收鍵盤事件
        self.setFocusPolicy(Qt.StrongFocus)
        
    def capture_screenshot(self):
        """擷取指定範圍的螢幕截圖，轉換為 OpenCV 格式"""
        screenshot = pyautogui.screenshot(region=(X1, Y1, X2 - X1, Y2 - Y1))
        screenshot = np.array(screenshot)
        self.image = cv2.cvtColor(screenshot, cv2.COLOR_RGB2BGR)
        self.display_image()
        print("截圖完成！")
        
    def run_yolo_detection(self):
        """執行 YOLO 偵測並計時"""
        if self.image is None:
            print("請先截圖！")
            return
        
        # 開始計時
        start_time = time.time()
        print("開始辨識中...")
        
        # 禁用按鈕，避免重複點擊
        self.detect_button.setEnabled(False)
        self.detect_button.setText("辨識中...")
        
        # 更新狀態
        self.status_label.setText("辨識中...")
        
        # 儲存臨時圖片
        cv2.imwrite("temp_screenshot.jpg", self.image)  # 暫存圖片供 YOLO 使用
        
        # 執行YOLO偵測
        result = subprocess.run([
            "python", "detect.py",
            "--img", "640",
            "--conf", "0.3",
            "--device", "0",
            "--weights", weights_path,
            "--source", "temp_screenshot.jpg",
            "--save-txt"
        ], capture_output=True, text=True)
        
        # 計算經過的時間
        elapsed_time = time.time() - start_time
        print(f"辨識完成！耗時 {elapsed_time:.2f} 秒")
        
        # 載入偵測結果
        self.load_detection_results()
        
        # 重新啟用按鈕
        self.detect_button.setEnabled(True)
        self.detect_button.setText("開始辨識")
         
    def load_detection_results(self):
        """載入 YOLO 偵測結果，存入全域變數 particle_data"""
        global particle_data  # 確保變數是全域的
        
        # 尋找最新的實驗資料夾
        exp_folders = glob.glob("runs/detect/exp*")
        if not exp_folders:
            print("未找到任何偵測結果資料夾！")
            return
        
        # 根據資料夾修改時間排序，取最新的
        latest_exp = max(exp_folders, key=os.path.getmtime)
        result_path = os.path.join(latest_exp, "labels", "temp_screenshot.txt")
        
        try:
            with open(result_path, "r") as f:
                detections = []
                for i, line in enumerate(f.readlines()):
                    values = line.strip().split()
                    if len(values) < 5:
                        continue
                    class_id = int(values[0])
                    x_center, y_center, width, height = map(float, values[1:5])
                    detections.append({
                        "particle_id": i + 1,
                        "class_id": class_id,
                        "x_center": x_center,
                        "y_center": y_center,
                        "width": width,
                        "height": height
                    })
            
            # 更新全域變數 particle_data
            particle_data = detections  
    
            # UI 內部變數也同步更新
            self.particle_data = detections  
    
            # 更新 UI 狀態標籤
            particles_count = len(detections)
            self.status_label.setText(f"辨識到 {particles_count} 個物體，已存入 particle_data")
            
            print(f"從 {result_path} 載入偵測結果")
            print("偵測完成！", particle_data)
        except FileNotFoundError:
            print(f"未找到偵測結果檔案：{result_path}")
            self.status_label.setText("未找到偵測結果檔案")
            
    def initUI(self):
        """建立 UI 介面"""
        self.setWindowTitle("粒子選擇器")
        self.setGeometry(100, 100, 800, 600)
        
        # 主 layout
        main_layout = QVBoxLayout()
        
        # 顯示影像
        self.image_label = QLabel(self)
        main_layout.addWidget(self.image_label)
        
        # 按鈕區域
        buttons_layout = QHBoxLayout()
        
        # 截圖按鈕
        self.capture_button = QPushButton("截圖 (空白鍵)", self)
        self.capture_button.clicked.connect(self.capture_screenshot)
        buttons_layout.addWidget(self.capture_button)
        
        # 開始辨識按鈕
        self.detect_button = QPushButton("開始辨識", self)
        self.detect_button.clicked.connect(self.run_yolo_detection)
        buttons_layout.addWidget(self.detect_button)
        
        main_layout.addLayout(buttons_layout)
        
        # 添加說明標籤
        instructions = QLabel("使用說明：\n1. 按下空白鍵或點擊「截圖」按鈕擷取螢幕\n2. 點擊「開始辨識」按鈕執行 YOLO 偵測")
        main_layout.addWidget(instructions)
        
        # 新增狀態標籤
        self.status_label = QLabel("就緒")
        self.status_label.setAlignment(Qt.AlignCenter)
        # 設定標籤樣式
        self.status_label.setStyleSheet("font-size: 14px; color: blue; margin-top: 10px;")
        main_layout.addWidget(self.status_label)
        
        self.setLayout(main_layout)
        
    def display_image(self):
        """顯示擷取的影像"""
        if self.image is None:
            return
        
        image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        height, width, channel = image.shape
        bytes_per_line = 3 * width
        q_image = QImage(image.data, width, height, bytes_per_line, QImage.Format_RGB888)
        self.image_label.setPixmap(QPixmap.fromImage(q_image))
        
    def keyPressEvent(self, event):
        """當使用者按下空白鍵時重新截圖"""
        if event.key() == Qt.Key_Space:
            print("空白鍵被按下")
            self.capture_screenshot()
        # 新增 ESC 鍵關閉程式
        elif event.key() == Qt.Key_Escape:
            self.close()

if __name__ == "__main__":
    weights_path = "C:/Users/Admin/Desktop/yolov9/yolov9/runs/train/exp5/weights/best.pt"
    app = QApplication(sys.argv)
    window = ParticleSelector()
    window.show()
    sys.exit(app.exec_())

截圖完成！
開始辨識中...
辨識完成！耗時 9.11 秒
從 runs/detect\exp59\labels\temp_screenshot.txt 載入偵測結果
偵測完成！ [{'particle_id': 1, 'class_id': 0, 'x_center': 0.237637, 'y_center': 0.972222, 'width': 0.0159341, 'height': 0.0277778}, {'particle_id': 2, 'class_id': 0, 'x_center': 0.0593407, 'y_center': 0.915741, 'width': 0.0120879, 'height': 0.0203704}, {'particle_id': 3, 'class_id': 0, 'x_center': 0.59478, 'y_center': 0.178241, 'width': 0.00934066, 'height': 0.012037}, {'particle_id': 4, 'class_id': 0, 'x_center': 0.726374, 'y_center': 0.410648, 'width': 0.0120879, 'height': 0.0138889}, {'particle_id': 5, 'class_id': 0, 'x_center': 0.839835, 'y_center': 0.0273148, 'width': 0.0115385, 'height': 0.0194444}, {'particle_id': 6, 'class_id': 0, 'x_center': 0.136264, 'y_center': 0.429167, 'width': 0.010989, 'height': 0.0157407}, {'particle_id': 7, 'class_id': 0, 'x_center': 0.929945, 'y_center': 0.0277778, 'width': 0.017033, 'height': 0.0185185}, {'particle_id': 8, 'class_id': 0, 'x_center': 0.581593, 'y_center': 0

## 測試截圖位置



In [5]:
import pyautogui
import time
from PIL import Image

def capture_specific_area(x1, y1, x2, y2, output_filename="screenshot.png"):
    """
    截取螢幕上指定座標範圍的區域
    
    參數:
    x1, y1: 左上角座標
    x2, y2: 右下角座標
    output_filename: 輸出檔案名稱
    """
    # 計算寬度和高度
    width = x2 - x1
    height = y2 - y1
    
    # 截取指定區域的螢幕
    screenshot = pyautogui.screenshot(region=(x1, y1, width, height))
    
    # 儲存截圖
    screenshot.save(output_filename)
    print(f"已截取 ({x1}, {y1}) 到 ({x2}, {y2}) 的區域，並保存為 {output_filename}")
    
    return screenshot

if __name__ == "__main__":
    # 設定要截取的區域座標
    x1, y1 = 100, 100  # 左上角座標
    x2, y2 = 500, 400  # 右下角座標
    
    # 延遲3秒，給使用者時間準備
    print("將在3秒後截圖...")
    time.sleep(3)
    
    # 執行截圖
    capture_specific_area(x1, y1, x2, y2, "my_screenshot.png")

將在3秒後截圖...
已截取 (100, 100) 到 (500, 400) 的區域，並保存為 my_screenshot.png


In [4]:
!pip install pyautogui


  Using cached PyAutoGUI-0.9.54-py3-none-any.whl
  Using cached PyMsgBox-1.0.9-py3-none-any.whl
  Using cached pytweening-1.2.0-py3-none-any.whl
  Using cached PyScreeze-1.0.1-py3-none-any.whl
  Using cached PyGetWindow-0.0.9-py3-none-any.whl
  Using cached MouseInfo-0.1.3-py3-none-any.whl
  Using cached PyRect-0.2.0-py2.py3-none-any.whl
  Using cached pyperclip-1.9.0-py3-none-any.whl


In [4]:
print(particle_data)

[{'particle_id': 1, 'class_id': 0, 'x_center': 0.456593, 'y_center': 0.161574, 'width': 0.0153846, 'height': 0.0194444}, {'particle_id': 2, 'class_id': 0, 'x_center': 0.658791, 'y_center': 0.207407, 'width': 0.0120879, 'height': 0.0185185}, {'particle_id': 3, 'class_id': 0, 'x_center': 0.380769, 'y_center': 0.208333, 'width': 0.00989011, 'height': 0.0166667}, {'particle_id': 4, 'class_id': 0, 'x_center': 0.885165, 'y_center': 0.972685, 'width': 0.010989, 'height': 0.0231481}, {'particle_id': 5, 'class_id': 0, 'x_center': 0.388187, 'y_center': 0.237963, 'width': 0.0126374, 'height': 0.0166667}, {'particle_id': 6, 'class_id': 0, 'x_center': 0.631868, 'y_center': 0.602315, 'width': 0.010989, 'height': 0.0157407}, {'particle_id': 7, 'class_id': 0, 'x_center': 0.766484, 'y_center': 0.292593, 'width': 0.010989, 'height': 0.0166667}, {'particle_id': 8, 'class_id': 0, 'x_center': 0.237637, 'y_center': 0.971759, 'width': 0.0159341, 'height': 0.0268519}, {'particle_id': 9, 'class_id': 0, 'x_cent

In [5]:
type(particle_data)

list

## 畫圖


In [4]:
import os
HOME = os.getcwd()
print (HOME)

C:\Users\Admin\Desktop\yolov9


In [5]:
%cd {HOME}/yolov9

C:\Users\Admin\Desktop\yolov9\yolov9


In [6]:
!python origin_version_virtual_complete_process_2025.py

Traceback (most recent call last):
  File "C:\Users\Admin\Desktop\yolov9\yolov9\origin_version_virtual_complete_process_2025.py", line 527, in <module>
    process_particle_data(particle_data, size, columns, rows, step_size)
                          ^^^^^^^^^^^^^
NameError: name 'particle_data' is not defined


In [4]:
from origin_version_virtual_complete_process_2025 import process_particle_data

# 假設你已經在 Jupyter 中有 particle_data 變數
process_particle_data(particle_data)


檢查路徑是否與障礙物重疊...
路徑 10 與障礙物重疊
路徑規劃處理時間: 0.00 秒


(array([[[255, 191,   0],
         [255, 191,   0],
         [255, 191,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[255, 191,   0],
         [255, 191,   0],
         [255, 191,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[255, 191,   0],
         [255, 191,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        ...,
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
  